## I'm trying to find Hartree Fock SCF Energies
### It's been kinda difficult not gonna lie 
by Jordan Ehrman 

Relied heavily on: https://medium.com/analytics-vidhya/practical-introduction-to-hartree-fock-448fc64c107b -- I actually think they did it right. 

Comments are all mine though. It's not working right now (jun 25). It'll run, but the energies and predicted bond lengths are all messed up.

In [18]:
# Here I'm importing a bunch of stuff.
# Numpy is used mostly to store things as arrays, 
# which allows me to take the norm of a vector and 
# easily subtract two lists, etc
import numpy as np 
from numpy import *
# And I think we only use scipy because it's easier to define the
# Boys function as an error function than it is to define it as 
# an integral
import scipy
from scipy.special import erf

In [19]:
# just defining the molecule. I don't have the coordinates here
# because I want to put them in a for loop down below 
# -- I'll be satisfied with this when it can at least 
# predict a bond distance tbh. 

# The medium article had a little bit more lines in here, 
# but I think some of them were redundant. 

number_of_atoms = 2

atoms = ['He','H']


# this is the sum of the max quantum numbers of the atoms in my system.
# if I wanted to generalize this, I'd put it in a loop. 
# But for now this is fine. 
B = 2

# Number of electrons in the system. 
N = 2

# nuclear charges. I... am not positive why the medium article uses these charges sometimes 
# and the zeta charges other times. Hopefully that'll click for me by the time I reach the end. 
charge_dict = {'H':1,'He':2}

In [20]:
# Defining my basis set. Took the numbers from the medium article, which in turn 
# took them from either Szabo or Pople.

STOnG = 3

#at first I thought these were effective nuclear charges -- whoops! 
#These are factors that allows my sum of gaussians to 
#approximate slater orbitals well. 
zeta_dict = {'H':[1.24],'He':[2.0925]}

max_quantum_number = {'H':1,'He':1}

#Standard basis function for hydrogen, standard basis function for helium. Making 
#my gauss+gauss+gauss look as much like a slater as possible. 
D = np.array([[0.44635,0.535328,0.154329]])
alpha = np.array([[0.109818,0.405771,2.22766]])

In [21]:
# Okay, so gaussians can be defined using their center and their 
# exponent (which is a scalar factor multiplied by the 'r**2' in the
# exponent part of the gaussian. Szabo calls it alpha -- are they alpha
# in the gaussians and zeta in the slater always? Not to be confused with 
# effective nuclear charge zeta. 
def gauss_product(gauss_A, gauss_B):
    Aalpha, Ra = gauss_A
    Balpha, Rb = gauss_B
    #if you multiply two exponential functions, the exponent gets summed.
    #Szabo calls this p, so I'll call it P. 
    p = Aalpha + Balpha
    #just the squared difference between the two centers. 
    diff = np.linalg.norm(Ra-Rb)**2
    #So, this one looks different than the one in Szabo. The author of 
    #the medium article explains that he normalizes it in this step -- 
    #if it doesn't work out for me, this is one place I want to check  
    #in desmos for mistakes
    Norm = (4*Aalpha*Balpha/(pi**2))**0.75
    # Actually completing the proportionality constant -- now renormalized.
    K = Norm*exp(-Aalpha*Balpha/p*diff)
    #The new center, straight from the book. 
    Rp = (Aalpha*Ra + Balpha*Rb)/p
    return p, diff, K, Rp

In [22]:
def overlap(A,B):
    p, diff, K, Rp = gauss_product(A,B)
    #this is straight from page 412 of Szabo. 
    return (pi/p)**1.5*K

In [23]:
def kinetic(A,B):
    #likewise straight from page 412.
    p, diff, K, Rp = gauss_product(A, B)
    Aalpha, Ra = A
    Balpha, Rb = B
    return Aalpha*Balpha/p*(3-2*Aalpha*Balpha/p*diff)*(pi/p)**1.5*K

In [24]:
def Fo(t):
    #I stole this format from medium, but it's on page 415 from szabo. 
    #The if else statement is to avoid errors from dividing by zero.
    if t==0:
        return 1
    else:
        return (0.5*(pi/t)**0.5)*erf(t**0.5)

In [25]:
def potential(A,B,atom_idx):
    #Also on 415 from szabo, but took the function from medium. 
    p,diff,K,Rp = gauss_product(A,B)
    Rc = atom_coordinates[atom_idx]
    Zc = charge_dict[atoms[atom_idx]]
    #I will admit, I had a typo here. I fixed it, and it didn't solve my problem. 
    return (-2*pi*Zc/p)*K*Fo(p*(np.linalg.norm(Rp-Rc))**2)

In [26]:
def multi(A,B,C,D):
    #also 415 of szabo -- the two electron integral. 
    p, diff_ab, K_ab, Rp = gauss_product(A,B)
    q, diff_cd, K_cd, Rq = gauss_product(C,D)
    #took this from medium -- some vast simplifications he must've done using 
    #algebra. much prefer this to szabo's rendition. But, if all else fails, 
    #this may be where there's a mistake. 
    multi_prefactor = 2*pi**2.5*(p*q*(p+q)**0.5)**-1
    return multi_prefactor*K_ab*K_cd*Fo(p*q/(p+q)*np.linalg.norm(Rp-Rq)**2)

In [27]:
def SD_successive_density_matrix_elements(Ptilde,P):
    #stolen from medium -- checking convergence by looping through elements
    #of the density matrix and taking their root mean squared difference. 
    x=0
    for i in range(B):
        for j in range(B):
            x += B**-2*(Ptilde[i,j]-P[i,j])**2
    return x ** 0.5

In [28]:
def get_nuclear_repulsion(): 
    #last function. It just tells you that the nuclear repulsion is 
    #equal to the product of the nuclear charges divided by their distance. 
    Nuc_repuls = 0 
    for idx_a, A in enumerate(atoms):
        for idx_b, B in enumerate(atoms):
            if idx_a == idx_b:
                continue
            charge_A = charge_dict[A]
            charge_B = charge_dict[B]
            product = charge_A*charge_B
            Ra = atom_coordinates[idx_a]
            Rb = atom_coordinates[idx_b]
            R = np.linalg.norm(Ra-Rb)
            Nuc_repuls += product/R
    return Nuc_repuls*0.5

In [29]:
listofenergies = []
#for coordchanger in [coordman * 0.01 for coordman in range(5, 400)]:
atom_coordinates = [np.array((0.0,0.0,0.0)),np.array((0.0,0.0, 1.4632))]
#okay, I took a lot of this from medium, but I'm going to try to comment it up and 
#move things around as I see fit to make sure I get what this is getting at. 
#starting with empty overlap and core hamiltonian parts. 
S = np.zeros((B,B))
T = np.zeros((B,B))
V = np.zeros((B,B))
multi_electron_tensor = np.zeros((B,B,B,B))
#looping through atoms
for idx_a, val_a in enumerate(atoms):
    #getting charge and location of nucleus of atom a
    Za = charge_dict[val_a]
    Ra = atom_coordinates[idx_a]
    #giving atom a an STO-3G orbital
    for m in range(max_quantum_number[val_a]):
        d_vec_m = D[m]
        zeta = zeta_dict[val_a][m]
        alpha_vec_m = alpha[m]*zeta**2
    #three times... 
    for p in range(STOnG):
        #for atom b...
        for idx_b, val_b in enumerate(atoms):
            #get nuclear charge and position 
            Zb = charge_dict[val_b]
            Rb = atom_coordinates[idx_b]
            for n in range(max_quantum_number[val_b]):
                #give it an STO-3G orbital
                d_vec_n = D[n]
                zeta = zeta_dict[val_b][n]
                alpha_vec_n = alpha[n]*zeta**2
                for q in range(STOnG):
                    a = (idx_a+1)*(m+1)-1
                    b = (idx_b+1)*(n+1)-1
                    #so here, we get the overlap matrix by taking the sum of the 
                    #overlaps of all nine combinations of gaussians. 
                    S[a,b] += d_vec_m[p]*d_vec_n[q]*overlap((alpha_vec_m[p],Ra),(alpha_vec_n[q],Rb))
                    if (a==0 and b==0):
                        print('adding' + str(d_vec_m[p]*d_vec_n[q]*overlap((alpha_vec_m[p],Ra),(alpha_vec_n[q],Rb)))+'to S(0,0) from exp,cont indices ' + str(p)+','+str(q))
                    #and here, we do the same to get kinetic energy
                    T[a,b] += d_vec_m[p]*d_vec_n[q]*kinetic((alpha_vec_m[p],Ra),(alpha_vec_n[q],Rb))

                    for i in range(number_of_atoms):
                    #and here, we do the same to get potential relative to each nucleus;
                    #summing the two gives total potential
                        V[a,b] += d_vec_m[p]*d_vec_n[q]*potential((alpha_vec_m[p],Ra),(alpha_vec_n[q],Rb),i)
                    #loading up the multi electron tensor 
                    for idx_c, val_c in enumerate(atoms):
                        #grabbing charge and location of third nuclei
                        Zc = charge_dict[val_c]
                        Rc = atom_coordinates[idx_c]
                        for k in range(max_quantum_number[val_c]):
                            #grabbing alpha and contraction coeff of third STO-3G 
                            d_vec_k = D[k]
                            zeta = zeta_dict[val_c][k]
                            alpha_vec_k = alpha[k]*zeta**2
                            for r in range(STOnG):
                                for idx_d, val_d in enumerate(atoms):
                                    #grabbing charge and location of fourth nuclei
                                    Zd = charge_dict[val_d]
                                    Rd = atom_coordinates[idx_d]
                                    for l in range(max_quantum_number[val_d]):
                                        #grabbing alpha and contraction coeff of fourth STO-3G
                                        d_vec_l = D[l]
                                        zeta = zeta_dict[val_d][l]
                                        alpha_vec_l = alpha[l]*zeta**2
                                        for s in range(STOnG):
                                            #indexing I stole from medium
                                            c = (idx_c+1)*(k+1)-1
                                            d = (idx_d+1)*(l+1)-1
                                            #loading up the multi electron tensor
                                            multi_electron_tensor[a,b,c,d] += d_vec_m[p]*d_vec_n[q]*d_vec_k[r]*d_vec_l[s]*(
                                            multi((alpha_vec_m[p],Ra),
                                                 (alpha_vec_n[q],Rb),
                                                 (alpha_vec_k[r],Rc),
                                                 (alpha_vec_l[s],Rd))
                                            )

adding0.19922832250000003to S(0,0) from exp,cont indices 0,0
adding0.17705203745284656to S(0,0) from exp,cont indices 0,1
adding0.0189643780484526to S(0,0) from exp,cont indices 0,2
adding0.17705203745284656to S(0,0) from exp,cont indices 1,0
adding0.28657606758399995to S(0,0) from exp,cont indices 1,1
adding0.05069055118868471to S(0,0) from exp,cont indices 1,2
adding0.0189643780484526to S(0,0) from exp,cont indices 2,0
adding0.05069055118868471to S(0,0) from exp,cont indices 2,1
adding0.023817440240999996to S(0,0) from exp,cont indices 2,2


In [30]:
print(multi_electron_tensor)

[[[[1.31389267 0.44007579]
   [0.44007579 0.60918706]]

  [[0.44007579 0.17858926]
   [0.17858926 0.3138368 ]]]


 [[[0.44007579 0.17858926]
   [0.17858926 0.3138368 ]]

  [[0.60918706 0.3138368 ]
   [0.3138368  0.77860306]]]]


In [31]:
#we get the right symmetric orthogonalization transformation. 
#Szabo uses the canonical one, Medium uses the symmetric one, 
#so I'm going to try to make the symmetric one work for now. 
evalS,U = np.linalg.eig(S)
diagS = dot(U.T,dot(S,U))
diagS_minushalf = diag(diagonal(diagS)**-0.5)
X = dot(U,dot(diagS_minushalf,U.T))

In [32]:
X
#it matches the Xsymmetric from Szabo

array([[ 1.08835493, -0.25950884],
       [-0.25950884,  1.08835493]])

# okay, all the numbers match szabo up until this point. 
### or are within 1% difference anyways
## somewhere in the SCF algorithm, something is going awry. I'm going to try to figure out where medium went wrong with this. 

In [34]:
Hcore = T + V

In [37]:
P = np.zeros((B,B))
P_previous = np.zeros((B,B))
P_list = []
threshold = 100
while threshold > (10**-4): 
    print('*************************')
    print('loop restarting')
    print('*************************')
    G = np.zeros((B,B))
    for i in range(B):
        for j in range(B):
            for x in range(B):
                for y in range(B):
                    G[i,j] += P[x,y]*(multi_electron_tensor[i,j,y,x]-0.5*multi_electron_tensor[i,x,y,j])
    print('Gmat: ', G)
    Fock = Hcore+G
    energy = 0
    for i in range(B):
        for j in range(B):
            energy += (1/2) * P[i,j] * (Hcore[i,j] + Fock[i,j])
    print('energy?:', energy)
    Fockprime = dot(X.T,dot(Fock,X))
    print('fockprime:', Fockprime)
    evalFockprime, Cprime = np.linalg.eig(Fockprime)
    print('coeffs:',Cprime)
    print('eigens:',evalFockprime)
    idx = evalFockprime.argsort()
    evalFockprime = evalFockprime[idx]
    Cprime = Cprime[:,idx]

    C = dot(X,Cprime)
    print('Coeffs of basis:', C)
    for i in range(B):
        for j in range(B):
            for a in range(int(N/2)):
                P[i,j] = 2*C[i,a]*C[j,a]
    print('new P:', P)
    P_list.append(P)
    threshold = SD_successive_density_matrix_elements(P_previous,P)
    print('threshold:',threshold)
    P_previous = P.copy()

*************************
loop restarting
*************************
Gmat:  [[0. 0.]
 [0. 0.]]
energy?: 0.0
fockprime: [[-2.50531325 -0.45017609]
 [-0.45017609 -1.47314039]]
coeffs: [[-0.93637294  0.35100672]
 [-0.35100672 -0.93637294]]
eigens: [-2.67406528 -1.30438836]
Coeffs of basis: [[-0.92801676  0.62501695]
 [-0.13902284 -1.11019545]]
new P: [[1.72243023 0.25803106]
 [0.25803106 0.0386547 ]]
threshold: 0.8805424667554685
*************************
loop restarting
*************************
Gmat:  [[1.26519374 0.37559333]
 [0.37559333 0.99150642]]
energy?: -4.142448650944319
fockprime: [[-1.15206105 -0.61736258]
 [-0.61736258 -0.4256439 ]]
coeffs: [[-0.86806574  0.49644926]
 [-0.49644926 -0.86806574]]
eigens: [-1.50513247 -0.07257248]
Coeffs of basis: [[-0.81593066  0.76558373]
 [-0.31504228 -1.0735966 ]]
new P: [[1.33148569 0.51410531]
 [0.51410531 0.19850327]]
threshold: 0.27818035477053443
*************************
loop restarting
*************************
Gmat:  [[1.20416029 0.30

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

evalFockprime[0]

x = [coordman * 0.01 for coordman in range(5, 400)]
y = listofenergies 

plt.title('He-H+ HOMO energy + nuc repulsion')
plt.xlabel('Distance (au)')
plt.ylabel('Energy (au)')

plt.plot(x,y,'o',color='black')

In [ ]:
evalFockprime[0]
